# 30 — Refunds
**Purpose:** Quote refund and, if allowed, apply (dry-run by default).

This notebook handles refund requests by retrieving relevant SOPs, identifying applicable policy clauses, and generating compliant refund quotes with proper citations and policy validation.


## SOP Retrieval (TODO)

### Knowledge Base Search Strategy
- **kb.search**: Query refund policy documents with relevant keywords
- **fs.read**: Access complete SOP documents for detailed clause analysis
- **Context Matching**: Match customer situation to specific policy clauses

### Search Query Generation
Based on customer request, generate targeted search queries:

#### **Query Examples**
- **"refund policy damaged item within 30 days"**
- **"return policy defective product warranty"**
- **"refund eligibility delivered order not as described"**
- **"cancellation policy processing order"**
- **"exchange policy wrong model compatibility"**

### SOP Document Processing
- **Front-matter Parsing**: Extract policy metadata and clause definitions
- **Clause Identification**: Map customer situation to specific policy clauses
- **Effective Date Validation**: Ensure using latest policy version
- **Condition Evaluation**: Check if customer situation meets policy conditions

### Policy Clause Mapping
Map customer scenarios to specific policy clauses:

#### **RF-2.3: Full Refund for Damaged/Incorrect Items**
- **Conditions**: `order_status == delivered AND within_days(30) AND reason in ['damaged','not_as_described']`
- **Action**: `refund <= 100% to original payment method`
- **Requirements**: `["proof_of_purchase"]`

#### **RF-1.2: Compensation for Shipping Delays**
- **Conditions**: `order_status == shipped AND carrier_delay == true`
- **Action**: `offer voucher <= 15% OR partial refund <= 20%`
- **Requirements**: `[]`

### Citation Generation
Create canonical citations for all policy references:

#### **Citation Format**
```json
{
  "source_path": "shared/sops/refunds_policy_v2024-07-01.md",
  "heading": "RF-2.3: Full Refund for Damaged/Incorrect Items",
  "hash": "sha256:abc123...",
  "clause_id": "RF-2.3",
  "effective_date": "2024-07-01"
}
```


## Quote Planner (TODO)

### Refund Quote Generation
Generate compliant refund quotes based on policy analysis and order details.

#### **Quote Options Structure**
```json
{
  "refund_options": [
    {
      "id": "refund_opt_1",
      "type": "full_refund",
      "amount": 89.00,
      "percentage": 100.0,
      "clause_id": "RF-2.3",
      "eligibility": true,
      "requirements": ["proof_of_purchase"],
      "processing_time": "5-7 business days",
      "rationale": "Item damaged within 30 days of delivery",
      "needs_HITL": false
    },
    {
      "id": "refund_opt_2", 
      "type": "partial_refund",
      "amount": 45.00,
      "percentage": 50.0,
      "clause_id": "RF-1.2",
      "eligibility": true,
      "requirements": [],
      "processing_time": "3-5 business days",
      "rationale": "Alternative option for damaged item",
      "needs_HITL": false
    }
  ]
}
```

### Quote Calculation Logic

#### **Full Refund Scenarios**
- **Damaged Items**: 100% refund within 30 days (RF-2.3)
- **Not as Described**: 100% refund within 30 days (RF-2.3)
- **Defective Products**: 100% refund within warranty period
- **Wrong Items**: 100% refund for shipping errors

#### **Partial Refund Scenarios**
- **Minor Issues**: 20-50% refund for minor problems
- **Customer Dissatisfaction**: 25-75% refund based on severity
- **Shipping Delays**: 15-20% refund or voucher (RF-1.2)
- **Compatibility Issues**: 30-50% refund for compatibility problems

#### **Exchange Scenarios**
- **Wrong Model**: Free exchange for correct model
- **Upgrade Options**: Exchange with price difference
- **Size/Color Changes**: Free exchange within 30 days
- **Defective Replacement**: Free exchange for same model

### HITL Decision Points

#### **High-Value Refunds (>$500)**
- **Amount Threshold**: Automatic HITL review for large refunds
- **Risk Assessment**: Evaluate financial impact and fraud risk
- **Approval Required**: Human approval before processing

#### **Policy Edge Cases**
- **Ambiguous Situations**: Cases not clearly covered by policy
- **Multiple Clauses**: Situations matching multiple policy clauses
- **Exceptional Circumstances**: Cases requiring special consideration

#### **Customer History**
- **Frequent Returns**: Customers with high return rates
- **Suspicious Patterns**: Potential abuse or fraud indicators
- **VIP Customers**: High-value customers requiring special handling

### Quote Validation

#### **Policy Compliance Check**
- **Clause Verification**: Ensure quote matches applicable policy clause
- **Condition Validation**: Verify customer situation meets policy conditions
- **Amount Validation**: Confirm refund amount within policy limits
- **Timeline Validation**: Check refund request within time limits

#### **Business Logic Validation**
- **Order Status**: Verify order status allows refund
- **Payment Method**: Confirm refund method availability
- **Customer Eligibility**: Check customer refund eligibility
- **Inventory Impact**: Assess impact on inventory and costs


## Output (TODO)

### State Updates
Append refund proposals and citations to shared state for policy validation and execution.

#### **Proposed Actions Format**
```json
{
  "proposed_actions": [
    {
      "id": "refund_opt_1",
      "action": "orders.refund_apply",
      "args": {
        "order_id": "o_2001",
        "amount": 89.00,
        "mode": "original",
        "reason": "damaged_item",
        "clause_id": "RF-2.3"
      },
      "rationale": "Full refund eligible under RF-2.3 for damaged item within 30 days",
      "expected_outcome": "Customer satisfaction and policy compliance",
      "cost_latency": "medium",
      "confidence": 0.95,
      "needs_HITL": false,
      "policy_compliance": "verified"
    }
  ]
}
```

#### **Citations Format**
```json
{
  "citations": [
    {
      "source_path": "shared/sops/refunds_policy_v2024-07-01.md",
      "heading": "RF-2.3: Full Refund for Damaged/Incorrect Items",
      "hash": "sha256:abc123...",
      "clause_id": "RF-2.3",
      "effective_date": "2024-07-01",
      "relevant_text": "Order status: Delivered, Time window: Within 30 days of delivery, Reason: Damaged item or not as described"
    }
  ]
}
```

### Output Requirements

#### **Mandatory Output Fields**
- **refund_options**: Array of valid refund options with amounts and terms
- **policy_clause**: Specific policy clause that applies to the situation
- **eligibility_status**: Clear indication of refund eligibility
- **requirements**: List of requirements for refund processing
- **processing_timeline**: Expected processing and delivery timeline
- **citations**: Complete policy citations with clause references

#### **Optional Output Fields**
- **alternative_options**: Alternative solutions (exchange, credit, etc.)
- **escalation_path**: Path for complex or edge cases
- **customer_communication**: Suggested customer messaging
- **follow_up_actions**: Required follow-up steps

### Customer Communication

#### **Refund Approval Message**
```
"Based on our refund policy (RF-2.3), I can process a full refund of $89.00 to your original payment method since the item was damaged and delivered within 30 days. The refund will be processed within 5-7 business days and you should see the credit in your account within 10-14 business days. I've created ticket #REF-2024-001 for tracking purposes."
```

#### **Refund Denial Message**
```
"Unfortunately, your refund request doesn't meet our current policy requirements. The item was delivered more than 30 days ago, which is outside our standard refund window. However, I can offer you a 20% discount voucher for your next purchase or connect you with our technical support team if you're experiencing issues with the product."
```

#### **Partial Refund Message**
```
"Based on your situation, I can offer a partial refund of $45.00 (50% of the original amount) under policy clause RF-1.2. This reflects the minor nature of the issue while providing fair compensation. Would you like me to process this partial refund, or would you prefer to explore other options like a free exchange?"
```

### Quality Assurance

#### **Quote Validation Checklist**
- ✅ Policy clause correctly identified and applied
- ✅ Refund amount calculated accurately
- ✅ Customer eligibility verified
- ✅ Timeline requirements met
- ✅ Citations properly formatted
- ✅ HITL requirements correctly assessed
- ✅ Alternative options considered

#### **Compliance Verification**
- ✅ Policy compliance confirmed
- ✅ Audit trail established
- ✅ Customer communication approved
- ✅ Processing requirements documented
- ✅ Escalation paths identified
